In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.plotset import setup_plot, save_fig

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2024,16,'R')
session.load()

In [ ]:
laps = session.laps
drivers = session.drivers
drivers = [session.get_driver(i).Abbreviation for i in session.drivers]

In [ ]:
stints = laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

for driver in drivers[:8]:
    driver_stints = stints.loc[stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        compound_color = fastf1.plotting.get_compound_color(row["Compound"],
                                                            session=session)
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=compound_color,
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]


# plt.title("2024 Dutch Grand Prix Tire Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
comp_palette = {'PIA_1_MEDIUM': '#ffd12e',
                'LEC_1_MEDIUM': '#ffd12e',
                'PIA_2_HARD': '#f0f0ec',
                'PIA_3_HARD': '#f0f0ec',
                'LEC_2_HARD': '#f0f0ec'
}

In [ ]:
line_comp_palette = {
                        'PIA_1_MEDIUM': '#FF8000',
                        'LEC_1_MEDIUM': '#FF0000',
                        'PIA_2_HARD': "#FF8000",
                        'PIA_3_HARD': '#FF8000',
                        'LEC_2_HARD': '#FF0000'
    }

In [ ]:
lec_df = session.laps.pick_drivers('LEC').copy()
lec_df['LapEndTime'] = lec_df['Time'].dt.total_seconds()
lec_df = lec_df.set_index('LapNumber')['LapEndTime']

pia_df = session.laps.pick_drivers('PIA').copy()
pia_df['LapEndTime'] = pia_df['Time'].dt.total_seconds()
pia_df = pia_df.set_index('LapNumber')['LapEndTime']


In [ ]:
gap_to_lec = pia_df - lec_df

In [ ]:
setup_plot(xyticksize=20, axeslabel=24)

fig, ax = plt.subplots(2,1, figsize=(15,12), sharex=True)

for drv in drivers[:2]:
    # drv_color = plotting.get_driver_color(session=session,identifier=drv)
    drv_color = {'LEC': 'red', 'PIA': '#FF8000'}
    laps = session.laps.pick_drivers(drv).pick_quicklaps(1.05).copy()
    laps['LapTime'] = laps['LapTime'].dt.total_seconds()
    laps['Driver_Stint_Compound'] = laps.apply(lambda x: x['Driver'] + '_' + str(int(x['Stint'])) + '_' + x['Compound'], axis=1)
    sns.lineplot(data=laps, x='LapNumber', y='LapTime', linewidth=4,
                 hue='Driver_Stint_Compound', palette=line_comp_palette, legend=False, ax=ax[0])

    sns.lineplot(data=laps, x='LapNumber', y='LapTime', linewidth=0, marker='o', markersize=12, markeredgecolor=None,
                 hue='Driver_Stint_Compound', palette=comp_palette, legend=False, ax=ax[0])

ax2 = ax[1].twinx()

ax[1].axhline(linewidth=1,linestyle='--',color='w')
ax[1].plot(gap_to_lec[gap_to_lec < 0], color='#FF8000', lw=4)
ax[1].plot(gap_to_lec[gap_to_lec > -1.2].loc[35:], color='red', lw=4)
ax[0].set_ylabel('Lap Time (sec)', labelpad=20)
ax[1].set_ylabel('Gap to Leclerc (sec)')
ax[1].set_xlabel('Lap Number')
ax2.set_yticks([])
ax2.set_ylabel('..',color="#00000000",labelpad=55)
# fig.align_ylabels(ax)

ax[0].set_xlim(0,55)
ax[0].set_ylim(81.5,86.5)
ax[1].set_xlim(0,55)
ax[1].set_ylim(-25,25)

In [ ]:
save_fig(fig=fig,name='race_trace',loc='Post7')